In [ ]:
# 코덱스200 -> 구글 API를 이용하여 관련 뉴스 수집 및 NAVER 종목 토론실 수집
# 코덱스 200 관련 200개 기업 뉴스 수집 및 NAVER 종목 토론실 수집

In [1]:
# 코덱스 관련 뉴스(빅카인즈)

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import time

In [45]:
class Browser():
    def __init__(self, url, search_word):
        self.url = url
        self.search_word = search_word
        
        # 접속
        driver = webdriver.Chrome()

        driver.implicitly_wait(5)
        driver.get(url)

        # 로그인
        driver.find_element(By.CSS_SELECTOR, '#header > div.hd-top > div > div.login-area > button.btn-login.login-modal-btn.login-area-before').click()
        element = driver.find_element(By.CSS_SELECTOR, '#login-user-id')
        element.send_keys("아이디")
        element = driver.find_element(By.CSS_SELECTOR, '#login-user-password')
        element.send_keys("비밀번호")
        driver.find_element(By.CSS_SELECTOR, '#login-btn').click()

        # 검색어 입력
        element = driver.find_element(By.CSS_SELECTOR,"#total-search-key")
        element.send_keys(search_word)
        element.send_keys("\n")
        
        self.driver = driver

In [46]:
def Split_and_Total(self, start_date, end_date, date_type, date_unit):
    # date 단위에 따라 다운로드 할 날짜 분리하기
    if date_type == "Year":
        first_month_s = datetime(year=int(start_date[0:4]), month = 1, day = 1).date()
        first_month_e = first_month_s + relativedelta(years = date_unit) - relativedelta(days = 1)
        last_month_s = datetime(year=int(end_date[0:4]), month = 1, day = 1).date()
        last_month_e = last_month_s + relativedelta(years = date_unit) - relativedelta(days = 1)

        time_df = pd.DataFrame({"start" : pd.date_range(start=first_month_s, end=last_month_s, freq = str(date_unit) + "YS"),
                                "end" : pd.date_range(start=first_month_e, end=last_month_e, freq = str(date_unit) + "Y")})
    elif date_type == "Month":
        first_month_s = datetime(year=int(start_date[0:4]), month = int(start_date[5:7]), day = 1).date()
        first_month_e = first_month_s + relativedelta(months = date_unit) - relativedelta(days = 1)
        last_month_s = datetime(year=int(end_date[0:4]), month = int(end_date[5:7]), day = 1).date()
        last_month_e = last_month_s + relativedelta(months = date_unit) - relativedelta(days = 1)

        time_df = pd.DataFrame({"start" : pd.date_range(start=first_month_s, end=last_month_s, freq = str(date_unit) + "MS"),
                                "end" : pd.date_range(start=first_month_e, end=last_month_e, freq = str(date_unit) + "M")})
        
    time_df.start.iloc[0] = start_date
    time_df.end.iloc[-1] = end_date
    
    over_df = time_df.apply(lambda date: Download(self, date.start, date.end),axis=1)
    return over_df

def Download(self, start, end):    
    driver = self.driver
    
    # 날짜 검색
    driver.find_element(By.CSS_SELECTOR, '#collapse-step-1').click()
    driver.find_element(By.CSS_SELECTOR, '#collapse-step-1-body > div.srch-detail.v2 > div > div.tab-btn-wp1 > div.tab-btn-inner.tab1 > a').click()
    
    # 시작 날짜
    element = driver.find_element(By.CSS_SELECTOR, '#search-begin-date')
    element.send_keys(Keys.CONTROL + "A");
    element.send_keys(start.strftime('%Y-%m-%d'))
    
    # 마지막 날짜
    element = driver.find_element(By.CSS_SELECTOR, '#search-end-date')
    element.send_keys(Keys.CONTROL + "A");
    element.send_keys(end.strftime('%Y-%m-%d'))

    # 경제, 정치, IT_과학 부분만 수집
    driver.find_element(By.CSS_SELECTOR, '#collapse-step-1-body > div.srch-detail.v2 > div > div.tab-btn-wp2 > div.tab-btn-inner.tab3 > a').click()
    driver.find_element(By.CSS_SELECTOR, '#srch-tab3 > ul > li:nth-child(1) > div > span:nth-child(3) > label > span').click()
    driver.find_element(By.CSS_SELECTOR, '#srch-tab3 > ul > li:nth-child(2) > div > span:nth-child(3) > label > span').click()
    driver.find_element(By.CSS_SELECTOR, '#srch-tab3 > ul > li:nth-child(8) > div > span:nth-child(3) > label > span').click()

    # 적용하기 버튼
    driver.find_element(By.CSS_SELECTOR, '#search-foot-div > div.foot-btn > button.btn.btn-search.news-search-btn.news-report-search-btn').click()
    
    # 뉴스 건수 추출
    time.sleep(3)
    n = driver.find_element(By.CSS_SELECTOR, '#news-results-tab > div.data-result-hd.pc-only.paging-v2-wrp > h3 > span.total-news-cnt').text
    print("------------------------------------ [ {} ~ {} ] : {}개 ----------------------------------".format(start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d'), n))
    
    # 2만개가 넘는 경우 데이터가 잘려서 다운로드 되므로 이 상황을 방지
    over_lst = []
    if int(n.replace(',','')) > 20000:
        over_lst.append([start, end])
    else:
        # 엑셀 다운로드
        driver.implicitly_wait(10)
        driver.find_element(By.CSS_SELECTOR, '#collapse-step-3').click()
        driver.find_element(By.CSS_SELECTOR, '#analytics-data-download > div.btm-btn-wrp > button').click()
    
    time.sleep(5)
    return over_lst

def Quit(self):
    return self.driver.quit()

Browser.Split_and_Total = Split_and_Total
Browser.Download = Download
Browser.Quit = Quit

In [47]:
url = "https://www.bigkinds.or.kr/"
search_word = "코덱스 OR KODEX"
browser_kodex = Browser(url = url, 
                  search_word = search_word)
over_kodex = browser_kodex.Split_and_Total(start_date = "2017-07-01", 
                               end_date = "2023-12-31", 
                               date_type = "Year", 
                               date_unit = 1)

------------------------------------ [ 2017-07-01 ~ 2017-12-31 ] : 834개 ----------------------------------
------------------------------------ [ 2018-01-01 ~ 2018-12-31 ] : 2,339개 ----------------------------------
------------------------------------ [ 2019-01-01 ~ 2019-12-31 ] : 1,877개 ----------------------------------
------------------------------------ [ 2020-01-01 ~ 2020-12-31 ] : 2,334개 ----------------------------------
------------------------------------ [ 2021-01-01 ~ 2021-12-31 ] : 1,406개 ----------------------------------
------------------------------------ [ 2022-01-01 ~ 2022-12-31 ] : 1,949개 ----------------------------------
------------------------------------ [ 2023-01-01 ~ 2023-12-31 ] : 2,456개 ----------------------------------


In [48]:
url = "https://www.bigkinds.or.kr/"
search_word = "삼성전자"
browser_samsung = Browser(url = url, 
                  search_word = search_word)
over_samsung = browser_samsung.Split_and_Total(start_date = "2023-12-01", 
                               end_date = "2023-12-31", 
                               date_type = "Month", 
                               date_unit = 1)

------------------------------------ [ 2017-07-01 ~ 2017-07-31 ] : 6,261개 ----------------------------------
------------------------------------ [ 2017-08-01 ~ 2017-08-31 ] : 8,064개 ----------------------------------
------------------------------------ [ 2017-09-01 ~ 2017-09-30 ] : 4,544개 ----------------------------------
------------------------------------ [ 2017-10-01 ~ 2017-10-31 ] : 5,473개 ----------------------------------
------------------------------------ [ 2017-11-01 ~ 2017-11-30 ] : 5,020개 ----------------------------------
------------------------------------ [ 2017-12-01 ~ 2017-12-31 ] : 4,796개 ----------------------------------
------------------------------------ [ 2018-01-01 ~ 2018-01-31 ] : 5,681개 ----------------------------------
------------------------------------ [ 2018-02-01 ~ 2018-02-28 ] : 6,926개 ----------------------------------
------------------------------------ [ 2018-03-01 ~ 2018-03-31 ] : 4,866개 ----------------------------------
-------------------